# Author : Kartik B Bhargav

# Sequence Classification

# Task: Aspect-level Sentiment Classification

In [1]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [0]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [0]:
import _pickle as cPickle


#Load Data

In [0]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [0]:
aspect_path = '/drive/My Drive/Deep Learning Course/practice-5-data/aspect_level-sentiment/aspect_level/' 


vocab = read_pickle(aspect_path, 'all_vocab.pkl')

train_x = read_pickle(aspect_path, 'train_x.pkl')
train_y = read_pickle(aspect_path, 'train_y.pkl')
dev_x = read_pickle(aspect_path, 'dev_x.pkl')
dev_y = read_pickle(aspect_path, 'dev_y.pkl')
test_x = read_pickle(aspect_path, 'test_x.pkl')
test_y = read_pickle(aspect_path, 'test_y.pkl')

train_aspect = read_pickle(aspect_path, 'train_aspect.pkl')
dev_aspect = read_pickle(aspect_path, 'dev_aspect.pkl')
test_aspect = read_pickle(aspect_path, 'test_aspect.pkl')


pretrain_data = read_pickle(aspect_path, 'pretrain_data.pkl')
pretrain_label = read_pickle(aspect_path, 'pretrain_label.pkl')

In [0]:
class Dataiterator_doc():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y
class Dataiterator_aspect():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, aspect_data, seq_length=32, decoder_dim=300, batch_size=32):
        
        len_aspect_data = len(aspect_data[0])
        #self.len_doc_data = len(doc_data[0])
        
        self.X_aspect = aspect_data[0] 
        self.y_aspect = aspect_data[1]
        self.aspect_terms = aspect_data[2]  
        self.num_data = len_aspect_data
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X_aspect = self.X_aspect[np.array(X_ids)] # X values (encoder input) per batch
        batch_y_aspect = self.y_aspect[np.array(X_ids)] # y_in values (decoder input) per batch
        batch_aspect_terms = self.aspect_terms[np.array(X_ids)]
        
        return batch_X_aspect, batch_y_aspect, batch_aspect_terms

          
    def all(self): # return all data examples
        return self.X_aspect, self.y_aspect, self.aspect_terms

In [7]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Lambda, Dropout, LSTM,Bidirectional
from keras.layers import Reshape, Activation, RepeatVector, concatenate, Concatenate, Dot, Multiply
import keras.backend as K
from keras.engine.topology import Layer
from keras import initializers
from keras import regularizers
from keras import constraints


Using TensorFlow backend.


In [0]:
overal_maxlen = 82
overal_maxlen_aspect = 7


#Define Attention Network Layer
- Define class for Attention Layer
- Calculating the attention weights

In [0]:

class Attention(Layer):
    def __init__(self,  **kwargs):
        """
        Keras Layer that implements an Content Attention mechanism.
        Supports Masking.
        """
       
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert type(input_shape) == list
       
        self.steps = input_shape[0][1]

        self.W = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),)

        self.built = True

    def compute_mask(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        assert type(mask) == list
        return None

    def call(self, input_tensor, mask=None):
        x = input_tensor[0]
        aspect = input_tensor[1]
        mask = mask[0]

        ###YOUR CODE HERE###

        y = K.transpose(K.dot(self.W, K.transpose(aspect)))
        y = K.expand_dims(y, axis=-2)
        y = K.repeat_elements(y, self.steps, axis=1)
        eij = K.sum(x * y, axis=-1)

        eij = K.tanh(eij)
        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        return a

   
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1])





In [0]:
class Average(Layer):
  
    def __init__(self, mask_zero=True, **kwargs):
        self.mask_zero = mask_zero
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x,mask=None):
        if self.mask_zero:           
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
            return K.sum(x, axis=1) / (K.sum(mask, axis=1) + K.epsilon())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def compute_mask(self, x, mask):
        return None

#Establish computation Graph for model
- Input tensors
- Shared WordEmbedding layer 
- Attention network layer  
- Shared BiLSTM layer
- Shared fully connected layer(prediction layer)


In [0]:
dropout = 0.5     
recurrent_dropout = 0.1
vocab_size = len(vocab)
num_outputs = 3 # labels

##Input tensors

In [12]:
#YOUR CODE HERE ##### Inputs #####
sentence_input = Input(shape=(overal_maxlen,), dtype='int32', name='sentence_input')
print(sentence_input)

aspect_input = Input(shape=(overal_maxlen_aspect,), dtype='int32', name='aspect_input')
print(aspect_input)

pretrain_input = Input(shape=(None,), dtype='int32', name='pretrain_input')
print(pretrain_input)

Tensor("sentence_input:0", shape=(None, 82), dtype=int32)
Tensor("aspect_input:0", shape=(None, 7), dtype=int32)
Tensor("pretrain_input:0", shape=(None, None), dtype=int32)


##Shared WordEmbedding layer

In [0]:
# word embedding layer
word_emb = Embedding(vocab_size, 300, mask_zero=True, name='word_emb')

In [0]:
# represent aspect as averaged word embedding ###
aspect_term_embs = word_emb(aspect_input)
aspect_embs = Average(mask_zero=True, name='aspect_emb')(aspect_term_embs)

In [0]:
## sentence representation from embedding ###
sentence_word_emb = word_emb(sentence_input)
pretrain_word_emb = word_emb(pretrain_input)

##Shared BiLSTM layer

In [16]:
## sentence representation from embedding ###
rnn = Bidirectional(LSTM(300, return_sequences=True, dropout=dropout, recurrent_dropout=recurrent_dropout, name='bilstm'))

sentence_lstm = rnn(sentence_word_emb)     # from aspect-level domain
pretrain_lstm = rnn(pretrain_word_emb)     # from document-level domain

print(sentence_lstm.shape)
print(pretrain_lstm.shape)

(None, 82, 600)
(None, None, 600)


##Attention Layer

In [17]:

attention_layer = Attention()([sentence_lstm, aspect_embs])
print(attention_layer.shape)

(None, 82)


##Prediction Layer

In [18]:

dot_sentence_attention = Dot(axes=1)([sentence_lstm, attention_layer])
dense_layer = Dense(3, activation='softmax')
aspect_probs = dense_layer(dot_sentence_attention)
print(aspect_probs.shape)

pretrain_avg = Average(mask_zero=True)(pretrain_lstm)
print(pretrain_avg.shape)

pretrain_probs = dense_layer(pretrain_avg)
print(pretrain_probs.shape)

(None, 3)
(None, 600)
(None, 3)


#Build Models for document-level and aspect-level data
- The two models shared the embedding, BiLSTM, Prediction Layer

In [19]:

model1 = Model(inputs=[pretrain_input], outputs=[pretrain_probs])
model2 = Model(inputs=[sentence_input, aspect_input], outputs=[aspect_probs])

model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_input (InputLayer)     (None, 82)           0                                            
__________________________________________________________________________________________________
aspect_input (InputLayer)       (None, 7)            0                                            
__________________________________________________________________________________________________
word_emb (Embedding)            multiple             3000900     aspect_input[0][0]               
                                                                 sentence_input[0][0]             
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) multiple             1442400     word_emb[1][0]             

#Train Model
- First Train model on document-level data.
- Then Train  model on aspect-level data

##Train on document-level data

In [0]:
import keras.optimizers as opt
optimizer=opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
batch_size = 128
train_steps_epoch = len(pretrain_data)/batch_size
batch_train_iter_doc = Dataiterator_doc(pretrain_data, pretrain_label, batch_size)

In [21]:

from keras.callbacks import EarlyStopping, ModelCheckpoint

def pretrain_generator(model, batch_train_iter_doc, pretrain_steps_epoch):
    
    earlystop_callbacks = [EarlyStopping(monitor='loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]
    
    def train_gen():
        while True:
            train_batches = [[X, y] for X, y in batch_train_iter_doc]
            for train_batch in train_batches:
                yield train_batch

    history = model.fit_generator(train_gen(), 
                                  steps_per_epoch=pretrain_steps_epoch,
                                  epochs = 5, 
                                  callbacks = earlystop_callbacks)

pretrain_generator(model1, batch_train_iter_doc, train_steps_epoch)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
235/234 [==============================] - 330s 1s/step - loss: 0.9422 - categorical_accuracy: 0.5457
Epoch 2/5
235/234 [==============================] - 328s 1s/step - loss: 0.7939 - categorical_accuracy: 0.6521
Epoch 3/5
235/234 [==============================] - 327s 1s/step - loss: 0.7435 - categorical_accuracy: 0.6687
Epoch 4/5
235/234 [==============================] - 324s 1s/step - loss: 0.7204 - categorical_accuracy: 0.6809
Epoch 5/5
235/234 [==============================] - 324s 1s/step - loss: 0.6604 - categorical_accuracy: 0.7207


##Train on aspect-level data

In [0]:
train_steps_epoch = len(train_x)/batch_size
batch_train_iter_aspect = Dataiterator_aspect([train_x, train_y, train_aspect], batch_size)
val_steps_epoch = len(dev_x)/batch_size
batch_val_iter_aspect = Dataiterator_aspect([dev_x, dev_y, dev_aspect], batch_size)

import keras.optimizers as opt
optimizer = opt.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=10, clipvalue=0)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [0]:

from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_generator(model, batch_train_iter, batch_val_iter, train_steps_epoch, val_steps_epoch):
    
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]
    
    def train_gen():
        while True:
            train_batches = [[[X, aspect], [y]] for X, y, aspect in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch
                
    def val_gen():
        while True:
            val_batches = [[[X, aspect], [y]] for X, y, aspect in batch_val_iter]
            for val_batch in val_batches:
                yield val_batch
                
    history = model.fit_generator(train_gen(), 
                                  validation_data=val_gen(),
                                  validation_steps=val_steps_epoch, 
                                  steps_per_epoch=train_steps_epoch,
                                  epochs = 10, 
                                  callbacks = earlystop_callbacks)


In [24]:
train_generator(model2, batch_train_iter_aspect, batch_val_iter_aspect, 
                train_steps_epoch, val_steps_epoch)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15/14 [===============================] - 7s 475ms/step - loss: 1.0481 - categorical_accuracy: 0.5417 - val_loss: 1.2943 - val_categorical_accuracy: 0.4844
Epoch 2/10
15/14 [===============================] - 6s 406ms/step - loss: 0.9935 - categorical_accuracy: 0.5958 - val_loss: 0.8800 - val_categorical_accuracy: 0.5625
Epoch 3/10
15/14 [===============================] - 6s 402ms/step - loss: 1.0004 - categorical_accuracy: 0.5771 - val_loss: 0.8317 - val_categorical_accuracy: 0.5781
Epoch 4/10
15/14 [===============================] - 6s 408ms/step - loss: 0.9786 - categorical_accuracy: 0.5604 - val_loss: 0.9809 - val_categorical_accuracy: 0.5938
Epoch 5/10
15/14 [===============================] - 6s 419ms/step - loss: 0.9204 - categorical_accuracy: 0.5833 - val_loss: 0.7491 - val_categorical_accuracy: 0.5859
Epoch 6/10
15/14 [===============================] - 6s 411ms/step - loss: 0.8426 - categorical_accuracy: 0.6167 - val_loss: 0.9410 - val_categorical_accuracy: 0.601

##Evaluating on test set
- show the accuracy

In [25]:
loss, accuracy = model2.evaluate([test_x, test_aspect], test_y)

print("Categorical accuracy: {:.4f}%".format(accuracy * 100))

638/638 [==============================] - 1s 2ms/step
Categorical accuracy: 58.4639%
